In [ ]:
data_path = "../../../Data/DMQA/cnn_stories/cnn/stories/"

In [ ]:
from os import listdir
files = listdir(data_path)
print(len(files))

_all_text = []
for file in files:
    with open(data_path + file, "r") as f:
        _all_text.append(f.read())

In [ ]:
from pytorch_pretrained_bert import BertTokenizer
import numpy as np
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
stride = 50
max_len = 450
max_tokenized_len = 511

In [ ]:
indices = np.random.randint(0, len(files), (100,))
lens = []
count = 0
dps = []
counter = 0
for text in _all_text:
    counter += 1
    split = text.index("@highlight")
    story = text[:split]
    summary = text[split:]
    summary = summary.replace("@highlight\n", "")
    summary = summary.replace("\n+", "\n")
    lines = np.asarray(summary.split("\n"))
    valid_lines = [len(line) > 5 for line in lines]
    summary = lines[valid_lines]
    lines = np.asarray(story.split("\n"))
    valid_lines = [len(line) > 5 for line in lines]
    story = lines[valid_lines]
    story = " ".join(story)
    summary = ". ".join(summary) + "."
    summary = summary.replace("NEW:", "")
    story = " ".join(story.split()[:max_len])

    _sto_tokens = tokenizer.tokenize(story)
    _sto_tokens = _sto_tokens[:max_tokenized_len]
    _sum_tokens = tokenizer.tokenize(summary)
    pointers = []
    for i in range(len(_sum_tokens)):
        token = _sum_tokens[i]
        if (any(char.isdigit() for char in token)):
            pointers.append(i)
    
    _story = tokenizer.convert_tokens_to_ids(_sto_tokens)
    _summary = tokenizer.convert_tokens_to_ids(_sum_tokens)
    dp = {
        "story":story,
        "summary":summary, 
        "story_tokens":_story, 
        "summary_tokens":_summary,
        "pointers":pointers
    }
    dps.append(dp)
    print(counter, "/", len(_all_text), end="\r")

In [ ]:
import pickle
pickle.dump(dps, open("../../../Data/DMQA/cnn_tokenized.pickle", "wb"))